Format lại code

In [1]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import math
import torch.multiprocessing as mp
from transformers import BertTokenizerFast
import pickle
# Evaluate
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.meteor_score import meteor_score
# from rogue import Rogue
# from pycocoevalcap.cider.cider import Cider
# from pycocoevalcap.spice.spice import Spice
#end
# mp.set_start_method('spawn', force=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
detokenize = tokenizer.convert_ids_to_tokens
batch_detokenize = tokenizer.batch_decode
BATCH_SIZE = 64 
#64 : 3.1GB VRAM b16 (3.0 Dedicated | 0.1 Shared)
device = 'cuda'
image_transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Dùng cho ImageNet
])


In [2]:
def load_image(image_folder_path: str): # DataLoader trên Jupyternotebook ko xử lý đa luồng đc nên đành phải load hết vô
    result = {}
    file_counts = len(os.listdir(image_folder_path))
    progress = 0
    last_log = 0
    count = 0
    for file_name in os.listdir(image_folder_path):
        file_path = os.path.join(image_folder_path, file_name)
        if os.path.isfile(file_path):
            id = os.path.splitext(os.path.basename(file_name))[0]
            if id in result: continue
            with Image.open(file_path).convert("RGB") as img:
                image = image_transforms(img)
                result[id] = torch.Tensor(image)
            count += 1
            progress = count / file_counts * 100
            if progress - last_log >= 10:
                last_log = progress
                print(f"Image loading {progress:.2f} % ")
    if last_log != 100:
        print(f"Image loading 100 % ")
    return result
def process_data(image_dict: dict[str, torch.Tensor], processed_data_path: str):
    result: list[tuple[torch.Tensor, torch.Tensor]] = []
    with open(processed_data_path, 'rb') as file:
        processed_data = pickle.load(file)
    for image_id, caption in processed_data:
        caption = torch.tensor(caption)
        image = image_dict[image_id]
        result.append((image, caption))
    return result
def get_train_test_loader(batch_size: int, n_wokers: int = 2):
    image_path = "../Flickr8k/Flicker8k_Dataset"
    train_path = "../Data_bert/train_set_bert.pkl"
    test_path = "../Data_bert/test_set_bert.pkl"
    image_dict: dict[str, torch.Tensor] = load_image(image_path)
    train_data = process_data(image_dict, train_path)
    test_data = process_data(image_dict, test_path)
    trainloader = DataLoader(train_data, batch_size=batch_size, num_workers=n_wokers, shuffle=True)
    testloader = DataLoader(test_data, batch_size=batch_size, num_workers=n_wokers, shuffle=False)
    return trainloader, testloader
trainloader, testloader = get_train_test_loader(BATCH_SIZE, 2)

Image loading 10.01 % 
Image loading 20.02 % 
Image loading 30.03 % 
Image loading 40.04 % 
Image loading 50.06 % 
Image loading 60.07 % 
Image loading 70.08 % 
Image loading 80.09 % 
Image loading 90.10 % 
Image loading 100 % 


In [3]:
# for images, captions in trainloader:
#     for i in range(min(4, captions.shape[0])):
#         print(images[i][:,100,100])
#         print(detokenize(captions[i]))
#     break

In [4]:
class EncoderCNN(nn.Module):
    def __init__(self, output_size: int):
        super(EncoderCNN, self).__init__()
        self.inception_model = models.inception_v3(pretrained=True)
        #self.inception_model.fc = torch.nn.Identity()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(1000, output_size)
        for name, param in self.inception_model.named_parameters():
            if "fc.weight" in name or "fc.bias" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
    def forward(self, images: torch.Tensor):
        features = self.inception_model(images) #[1, 2048]
        if isinstance(features, tuple):  # Nếu là tuple
            features = features[0] 
        features = self.relu(features)
        features = self.dropout(features)
        features = self.fc(features)
        return features
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, vocab_size, hidden_size, input_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size + input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.use_hidden = False
    def forward(self, features: torch.Tensor, captions: torch.Tensor, hidden_state: tuple[torch.Tensor, torch.Tensor] = None): 
        # Constant : seq = 1
        # features : [bsz, img_sz]
        # captions : [bsz, seq]
        # hidden : [num_layers, bsz, hidden]
        embeddings = self.embed(captions) # [bsz, seq, embed]
        features = features.unsqueeze(1).expand(-1, embeddings.shape[1], -1) # [bsz, seq, embed]
        combined = torch.cat((features, embeddings), dim=2) # [bsz, seq, img_sz + embed]
        # hidden_state : [num_layers, seq, hid] * 2
        if self.use_hidden:
            output, hidden_state = self.lstm(combined, hidden_state)
        else:
            output, hidden_state = self.lstm(combined)
        # output : [bsz, seq, vocab_size]
        output = self.relu(output)
        output = self.dropout(output)
        output = self.linear(output)
        return output, hidden_state
class ImageToTextModel(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module):
        super(ImageToTextModel, self).__init__()
        self.encoder: EncoderCNN = encoder
        self.decoder: DecoderRNN = decoder
    def forward(self, images: torch.Tensor, captions: torch.Tensor):
        # Constant : SEQ_LENGTH = 46, seq = 1
        # images: [bsz, 3, raw_image_width, raw_image_height]
        # captions: [bsz, SEQ_LENGTH]
        bsz = images.shape[0]
        if self.decoder.use_hidden:
            hidden_state: tuple[torch.Tensor, torch.Tensor] = None
        features = self.encoder(images)
        # features: [bsz, img_sz]
        seq_predicted = []
        seq_predicted.append(torch.zeros((bsz, self.decoder.vocab_size), dtype=torch.float32).unsqueeze(1).to(device))
        # seq_predicted : [predict_length, seq, vocab]
        decoder_input = captions[:, 0].unsqueeze(1)
        # decoder_input : [bsz, seq]
        seq_length = captions.shape[1]
        for di in range(1, seq_length):
            if self.decoder.use_hidden:
                output_decoder, hidden_state = self.decoder(features, decoder_input, hidden_state)
            else:
                output_decoder, hidden_state = self.decoder(features, decoder_input)
            # ouput_decoder: [bsz, seq, vocab]
            # hidden_state: [num_layers, bsz, hidden_size] * 2
            decoder_input = captions[:, di].unsqueeze(1)
            # decoder_input : [bsz, seq]
            seq_predicted.append(output_decoder)
        return torch.cat(seq_predicted, dim=1)
    def predict(self, images: torch.Tensor, captions: torch.Tensor, predict_length: int):
        bsz = images.shape[0]
        hidden_state: tuple[torch.Tensor, torch.Tensor] = None
        features = self.encoder(images)
        seq_predicted = []
        seq_predicted.append(torch.zeros((bsz, self.decoder.vocab_size), dtype=torch.float32).unsqueeze(1).to(device))
        decoder_input = captions
        for _ in range(predict_length-1):
            output_decoder, hidden_state = self.decoder(features, decoder_input, hidden_state)
            seq_predicted.append(output_decoder)
            decoder_input = output_decoder.argmax(2)
        return torch.cat(seq_predicted, dim=1)


In [5]:
def train(model : ImageToTextModel, dataloader : DataLoader, lossf : callable, optimizer : torch.optim.Optimizer, mixed: bool, device: str):
    model.train()
    epoch_loss = 0
    count = 0
    for images, captions in dataloader:
        optimizer.zero_grad()
        images: torch.Tensor = images.to(device)
        captions: torch.Tensor = captions.to(device)
        if mixed:
            with torch.autocast(device_type=device, dtype=torch.bfloat16):
                outputs: torch.Tensor = model(images, captions)
                loss: torch.Tensor = lossf(outputs.view(-1, outputs.shape[2]), captions.view(-1))
        else:
            outputs: torch.Tensor = model(images, captions)
            loss: torch.Tensor = lossf(outputs.view(-1, outputs.shape[2]), captions.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        count += 1 
    return epoch_loss / count
def compute_metrics(predict: torch.Tensor, caption: torch.Tensor): # Quên chưa xóa token đầu ...
    # Accuracy
    is_correct = 1 if predict[1] == caption[1] else 0
    caption: list[str] = detokenize(caption)
    predict: list[str] = detokenize(predict)
    while (caption[-1] == "[PAD]"):
        caption.pop()
    while (predict[-1] == "[PAD]"):
        predict.pop()
    # Bleu
    reference = [caption]
    # print(caption, predict)
    bleu_1 = sentence_bleu(reference, predict, weights=(1, 0, 0, 0))
    bleu_2 = sentence_bleu(reference, predict, weights=(0.5, 0.5, 0, 0))
    bleu_3 = sentence_bleu(reference, predict, weights=(0.33, 0.33, 0.33, 0))
    bleu_4 = sentence_bleu(reference, predict)
    # Rogue
    # rogue = Rogue()
    # scores = rogue.get_scores(' '.join(predicts), ' '.join(captions))
    # Meteor
    # meteor_score_ = meteor_score([' '.join(captions)], ' '.join(predicts))
    # Cider, spice
    # cider_scorer = Cider()
    # spice_scorer = Spice()
    # cider_score, _ = cider_scorer.compute_score({0 : [' '.join(caption)]}, {0 : [' '.join(predict)]})
    # spice_score, _ = spice_scorer.compute_score({0 : [' '.join(captions)]}, {0 : [' '.join(predicts)]})
    return {
        "accuracy" : is_correct,
        "bleu_1" : bleu_1,
        "bleu_2" : bleu_2,
        "bleu_3" : bleu_3,
        "bleu_4" : bleu_4,
        # "rogue-1" : scores["rogue-1"]["f"],
        # "rogue-2" : scores["rogue-2"]["f"],
        # "rogue-l" : scores["rogue-l"]["f"],
        # "meteor" : meteor_score_,
        # "cider" : cider_score,
        # "spice" : spice_score
    }
def test(model : ImageToTextModel, dataloader : DataLoader, lossf : callable, mixed: bool, device: str):
    model.eval()
    count = 0
    total_count = 0
    epoch_metrics = {'loss' : 0}
    input = torch.tensor([101]).to(device)
    for images, captions in dataloader:
        images: torch.Tensor = images.to(device)
        captions: torch.Tensor = captions.to(device)
        bsz = images.shape[0]
        inputs = input.unsqueeze(1).expand((bsz, 1))
        if mixed:
            with torch.autocast(device_type=device, dtype=torch.bfloat16):
                outputs: torch.Tensor = model.predict(images, inputs, captions.shape[1])
                loss: torch.Tensor= lossf(outputs.view(-1, outputs.shape[2]), captions.view(-1))
        else:
            outputs: torch.Tensor  = model.predict(images, inputs, captions.shape[1])
            loss: torch.Tensor= lossf(outputs.view(-1, outputs.shape[2]), captions.view(-1))
        predicts = outputs.argmax(2)
        count += 1
        total_count += bsz
        epoch_metrics['loss'] += loss.item()
        for i in range(bsz):
            metrics = compute_metrics(predicts[i], captions[i])
            for key in metrics:
                if key not in epoch_metrics:
                    epoch_metrics[key] = metrics[key]
                else:
                    epoch_metrics[key] += metrics[key]
    for key in epoch_metrics:
        if key in ['loss']:
            epoch_metrics[key] /= count
        else:
            epoch_metrics[key] /= total_count
    return epoch_metrics

In [6]:
import util
image_size = 256
embed_size = 256
hidden_size = 256
encoder = EncoderCNN(
    output_size=image_size
)
decoder = DecoderRNN(
    embed_size=embed_size,
    vocab_size=tokenizer.vocab_size,
    hidden_size=hidden_size,
    input_size=image_size,
    num_layers=1
)
decoder.use_hidden = True
image_to_text_model = ImageToTextModel(
    encoder=encoder,
    decoder=decoder
)
loss_func= nn.CrossEntropyLoss(ignore_index=0)
model_path, train_log = util.train_eval(
    trainloader=trainloader, # train data loader
    testloader=testloader, # test data loader
    model=image_to_text_model, # model
    train_func=train, # hàm train
    test_func=test, # hàm test
    lossf=loss_func, # hàm loss
    end_epoch=50, # số epoch kết thúc train (tăng lên khi train từ checkpoint nhá, ko phải số epoch nó chạy code mà số epoch nó sẽ dừng)
    lr=2.5e-3, # lr sau khi warmup
    gamma=0.95, # lr * gamma ^ epoch sau warmup
    log_step=1, # in ra console mỗi k epochs
    warmup_nepochs=10, # số epoch chạy warmup scheduler
    warmup_lr=1e-3, # lr ban đầu của warmup
    warmup_gamma=1.1, # lr * gamma ^ epoch khi chạy warmup
    save_weight=True, # lưu trọng số mô hình
    save_full=True, # lưu optimizer, scheduler để train
    save_each=1, # luu mỗi k epoch
    mixed_train = True, # sử dụng bf16 cho train
    mixed_eval = False, # sử dùng bf16 cho eval
    # load_checkpoint = True, # load checkpoint cũ ko
    # load_optimizer = True, # load optimizer cũ ko (sau khi load checkpoint)
    # checkpoint_path = "checkpoint/i256_e256_h256_model/2024-12-12_20-37-10", # folder path của checkpoint
    save_path="checkpoint/i256_e256_h_256_model", # folder path lưu mô hình
    optimizer_type=torch.optim.Adam, # loại optimizer sử dụng
    device=device, # cpu hay cuda
    metadata_extra={ # tham số lưu thêm vào metada khi lưu mô hình
        "batch_size" : BATCH_SIZE,
        "dataset_name" : "Flickr8k",
        "use_hidden" : decoder.use_hidden
    },
    log_metric=True # in các metrics ra cốnle
)

c:\Users\Anh\.conda\envs\data\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Anh\.conda\envs\data\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start train


  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\Anh\.conda\envs\data\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Anh\.conda\envs\data\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Anh\.conda\envs\data\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using l

Train loss : 6.0180 | Test loss : 5.7932 | Train time : 241.04 s | Lr : 0.00100000
{'loss': 5.793184793448146, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


  2%|▏         | 1/50 [04:50<3:57:04, 290.29s/it]

...

 96%|█████████▌| 48/50 [3:08:58<07:48, 234.39s/it]

Train loss : 5.7117 | Test loss : 5.7492 | Train time : 190.05 s | Lr : 0.00035599
{'loss': 5.749236342273181, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


100%|██████████| 50/50 [3:16:46<00:00, 236.13s/it]

Train loss : 5.7096 | Test loss : 5.7485 | Train time : 194.53 s | Lr : 0.00033819
{'loss': 5.748467173757432, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


Complete


In [7]:
import util
image_size = 256
embed_size = 64
hidden_size = 256
encoder = EncoderCNN(
    output_size=image_size
)
decoder = DecoderRNN(
    embed_size=embed_size,
    vocab_size=tokenizer.vocab_size,
    hidden_size=hidden_size,
    input_size=image_size,
    num_layers=1
)
decoder.use_hidden = True
image_to_text_model = ImageToTextModel(
    encoder=encoder,
    decoder=decoder
)
loss_func= nn.CrossEntropyLoss(ignore_index=0)
model_path, train_log = util.train_eval(
    trainloader=trainloader, # train data loader
    testloader=testloader, # test data loader
    model=image_to_text_model, # model
    train_func=train, # hàm train
    test_func=test, # hàm test
    lossf=loss_func, # hàm loss
    end_epoch=50, # số epoch kết thúc train (tăng lên khi train từ checkpoint nhá, ko phải số epoch nó chạy code mà số epoch nó sẽ dừng)
    lr=2.5e-3, # lr sau khi warmup
    gamma=0.95, # lr * gamma ^ epoch sau warmup
    log_step=1, # in ra console mỗi k epochs
    warmup_nepochs=10, # số epoch chạy warmup scheduler
    warmup_lr=1e-3, # lr ban đầu của warmup
    warmup_gamma=1.1, # lr * gamma ^ epoch khi chạy warmup
    save_weight=True, # lưu trọng số mô hình
    save_full=True, # lưu optimizer, scheduler để train
    save_each=1, # luu mỗi k epoch
    mixed_train = True, # sử dụng bf16 cho train
    mixed_eval = False, # sử dùng bf16 cho eval
    # load_checkpoint = True, # load checkpoint cũ ko
    # load_optimizer = True, # load optimizer cũ ko (sau khi load checkpoint)
    # checkpoint_path = "checkpoint/i256_e256_h256_model/2024-12-12_20-37-10", # folder path của checkpoint
    save_path="checkpoint/i256_e64_h_256_model", # folder path lưu mô hình
    optimizer_type=torch.optim.Adam, # loại optimizer sử dụng
    device=device, # cpu hay cuda
    metadata_extra={ # tham số lưu thêm vào metada khi lưu mô hình
        "batch_size" : BATCH_SIZE,
        "dataset_name" : "Flickr8k",
        "use_hidden" : decoder.use_hidden
    },
    log_metric=True # in các metrics ra cốnle
)

Start train


  0%|          | 0/50 [00:00<?, ?it/s]

Train loss : 6.0202 | Test loss : 5.7914 | Train time : 179.92 s | Lr : 0.00100000
{'loss': 5.791400951675222, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


  2%|▏         | 1/50 [03:40<3:00:17, 220.77s/it]

Train loss : 5.8250 | Test loss : 5.7859 | Train time : 183.47 s | Lr : 0.00110000
{'loss': 5.785941063603269, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


...

 96%|█████████▌| 48/50 [2:56:48<07:22, 221.42s/it]

Train loss : 5.7062 | Test loss : 5.7456 | Train time : 184.05 s | Lr : 0.00035599
{'loss': 5.745610496665858, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


100%|██████████| 50/50 [3:04:16<00:00, 221.13s/it]

Train loss : 5.7049 | Test loss : 5.7458 | Train time : 182.40 s | Lr : 0.00033819
{'loss': 5.745810194860531, 'accuracy': 0.615, 'bleu_1': 0.034013043478259536, 'bleu_2': 2.4360880535736356e-155, 'bleu_3': 2.5050322592968467e-204, 'bleu_4': 6.641550455034804e-232}


Complete


In [8]:
import util
image_size = 64
embed_size = 256
hidden_size = 256
encoder = EncoderCNN(
    output_size=image_size
)
decoder = DecoderRNN(
    embed_size=embed_size,
    vocab_size=tokenizer.vocab_size,
    hidden_size=hidden_size,
    input_size=image_size,
    num_layers=1
)
decoder.use_hidden = True
image_to_text_model = ImageToTextModel(
    encoder=encoder,
    decoder=decoder
)
loss_func= nn.CrossEntropyLoss(ignore_index=0)
model_path, train_log = util.train_eval(
    trainloader=trainloader, # train data loader
    testloader=testloader, # test data loader
    model=image_to_text_model, # model
    train_func=train, # hàm train
    test_func=test, # hàm test
    lossf=loss_func, # hàm loss
    end_epoch=50, # số epoch kết thúc train (tăng lên khi train từ checkpoint nhá, ko phải số epoch nó chạy code mà số epoch nó sẽ dừng)
    lr=2.5e-3, # lr sau khi warmup
    gamma=0.95, # lr * gamma ^ epoch sau warmup
    log_step=1, # in ra console mỗi k epochs
    warmup_nepochs=10, # số epoch chạy warmup scheduler
    warmup_lr=1e-3, # lr ban đầu của warmup
    warmup_gamma=1.1, # lr * gamma ^ epoch khi chạy warmup
    save_weight=True, # lưu trọng số mô hình
    save_full=True, # lưu optimizer, scheduler để train
    save_each=1, # luu mỗi k epoch
    mixed_train = True, # sử dụng bf16 cho train
    mixed_eval = False, # sử dùng bf16 cho eval
    # load_checkpoint = True, # load checkpoint cũ ko
    # load_optimizer = True, # load optimizer cũ ko (sau khi load checkpoint)
    # checkpoint_path = "checkpoint/i256_e256_h256_model/2024-12-12_20-37-10", # folder path của checkpoint
    save_path="checkpoint/i256_e64_h_256_model", # folder path lưu mô hình
    optimizer_type=torch.optim.Adam, # loại optimizer sử dụng
    device=device, # cpu hay cuda
    metadata_extra={ # tham số lưu thêm vào metada khi lưu mô hình
        "batch_size" : BATCH_SIZE,
        "dataset_name" : "Flickr8k",
        "use_hidden" : decoder.use_hidden
    },
    log_metric=True # in các metrics ra cốnle
)

Start train


  0%|          | 0/50 [00:00<?, ?it/s]

Train loss : 5.3344 | Test loss : 6.4149 | Train time : 197.46 s | Lr : 0.00100000
{'loss': 6.414938926696777, 'accuracy': 0.615, 'bleu_1': 0.09072173913043691, 'bleu_2': 0.04467780976477441, 'bleu_3': 0.006383117533577904, 'bleu_4': 0.0023118513968456464}


  2%|▏         | 1/50 [03:59<3:15:40, 239.61s/it]

Train loss : 4.1432 | Test loss : 7.0560 | Train time : 190.65 s | Lr : 0.00110000
{'loss': 7.056033279322371, 'accuracy': 0.615, 'bleu_1': 0.10394347826087183, 'bleu_2': 0.057783027093006444, 'bleu_3': 0.007852922847774526, 'bleu_4': 0.002917439640644708}


...

 96%|█████████▌| 48/50 [3:24:47<08:44, 262.35s/it]

Train loss : 2.4452 | Test loss : 11.3913 | Train time : 225.83 s | Lr : 0.00035599
{'loss': 11.391299296029006, 'accuracy': 0.6152, 'bleu_1': 0.11505217391304588, 'bleu_2': 0.06702450582824769, 'bleu_3': 0.01788659174830984, 'bleu_4': 0.007419327717471376}


100%|██████████| 50/50 [3:33:48<00:00, 256.56s/it]

Train loss : 2.4400 | Test loss : 11.3976 | Train time : 219.52 s | Lr : 0.00033819
{'loss': 11.39760019809385, 'accuracy': 0.615, 'bleu_1': 0.11586956521739385, 'bleu_2': 0.06725602007077935, 'bleu_3': 0.01771069264615197, 'bleu_4': 0.007425141424463584}


Complete
